## Abstract

This research analyzed credit card debt defaults and explored five classifiers (AdaBoost, Logistic Regression, KNN, Random Forests, and Bagged Decision Trees (essentially random forest again)), and multiple data mining techniques in pursuit of a reliable method for discerning between high and low risk credit card users in a large, imbalanced data set. The target feature of the data set, default_payment $(1=\text{yes}, \; 0=\text{no})$, is imbalanced with about $22\%$ of the observed credit users defaulting in at least 1 of the 6 included months, so balancing techniques like SMOTE and SMOTETokem were explored with the 5 examined classifiers. For this business application, underestimating a cardholder's default risk can result in loss of liquidity or possible permanent loss of money, while overestimating a cardholder's credit risk can result in missed profit. As loss is much more costly than missed profit, I used recall and precision as the primary metrics for evaluating classifier performance, and I explored techniques for visualizing classifier performance, and I improved on sklearn's ROC curve visualization to present threshold information that enables an analyst to quickly evaluate the threshold to choose to identify the credit risks. 

Additionally, A classifier's performance metrics can vary greatly depending on the data preprocessing steps, so as a secondary goal, I sought to develop a more general framework for streamlining the combination of preprocessing steps, with the intention of adding that framework to my toolbox for use with future problems. 

### Objectives

The accuracy and performance of a machine learning classifier is highly dependent on the preprocessing steps applied to the data. With big data sets, inadequate data preprocessing or poor algorithm parameter selection can cause prohibitively long run times. My initial project focus was in exploring the performance of multiple base classifiers across multiple large ($n_{obs}>10,000$) with faint signals concerning financial risk. The time constraints of the project and the time costs of tuning classifiers revealed a more interesting project, namely to build a framework to streamline the process of testing different preprocessing configurations over a number of different base classifiers, and I used one large data set to evaluate this framework.  This involved experimenting with many different tools and attempting to adapt them to my use-case.

Additionally, I am very interested in visualizing data (I took Dr. Eli Brown's CSC595 on Interactive Data Visualization last quarter and I loved it. I'm still proud of [my final project](http://bl.ocks.org/MattTriano/raw/154e9c142504d61985e2b65287e389be/)), so I spent a lot of time improving the visualizations for precision-recall relationships. This was another significant time consumer, but I'm pleased with the results produced. 

### Risk Analysis

For a financial entity to maintain adequate financial reserves, that entity must be able to both accurately assess their exposure to risk. For a financial entity to make profitable loans, that entity must keep the proportion of bad loans (ie loans where the lendee fails to make payments on time) below some threshold proportion. Both of these goals require the financial entity to have a reasonably accurate method for discriminating between people who will default on that loan and people who will regularly make their payments on time.

### Data Set: Taiwanese Credit Card Data

This data set was initially collected by a published by 

<img src="final_proj/data_set.png" alt="Drawing" style="width: 900px;"/>

      Fig. 1.  Data Set Distributions

<img src="final_proj/pay_dists.png" alt="Drawing" style="width: 900px;"/>

      Fig. 2.  Payment Delinquency Distributions by Month

### Data Set Feature Descriptions

|--- Feature Name ---|-------- Feature Description --------|------------------  Defined Values  ------------------|
| :--------------: | --------------------: | :---------------- |
| $\text{default_payment_next_month}$ | Binary Target Feature indicating a Default on Payment | $0=\text{No Default}$ |
|  |  | $1=\text{Default}$ |
| $LIMIT\_BAL$ | Ammount of available consumer and family credit [NT \$] | $LIMIT\_BAL \geq 0$ |
| $SEX$ | Gender  | $1=\text{Male}$ |
|  |  | $2=\text{Female}$ |
| $EDUCATION$ | Highest completed education level | $1=\text{Grad School}$ |
|  |  | $2=\text{University}$ |
|  |  | $3=\text{High School}$ |
|  |  | $4=\text{Other}$ |
| $MARRIAGE$ | Marital Status | $1=\text{Married}$ |
|  |  | $2=\text{Single}$ |
|  |  | $3=\text{Other}$ |
| $AGE$ | Age | $AGE \geq 0$ |
| $PAY\_x$ | Number of months delinquent 'x' months ago | $-1=\text{no delinquency}$ |
|  |  | $ 1\text{ up to }N=\text{Delinquent 1 to N months}$ |
| $BILL\_AMT\_x$ | Ammount of bill statement 'x' months ago [NT \$] | Any real number (negative values are credits) |
| $PAY\_AMT\_x$ | Amount paid 'x' months ago [NT \$] | $PAY\_AMT\_x \geq 0$ |

### Evaluation Metrics and Visualization Development

For this problem, it's far more important to identify False-Negatives and True-Positives than it is to identify False-Positives and True-Negatives, as False-Negatives involve underestimating a potential lendee's creditworthiness (thereby putting money in the hands of people less likely to pay it back) and True-Positives involve correctly identifying and avoiding making bad loans.  False-Positives involve overestimating the risk of a loan, and thereby forgoing the potential profit. This is also undesirable, but it is more of a business concern to lose money than to fail to gain profit. These qualities are excellently expressed in the **ROC curve**$^{[ROCbib]}$, which essentially shows how accurate a classifier is in discriminating between and ranking true positives vs true negatives. ROC curves plot the True-Positive Rate (**TPR**) and False-Positive Rate (**FPR**) for a classifier. For a binary classifier, the TPR and FPR can be calculated using values from that binary classifier's confusion matrix, which consists of True-Positives (**TP**, actually positive and predicted positive), False-Positives (**FP**, actually negative but predicted positive), True-Negatives (**TN**, actually negative and predicted negative), and False-Negatives (**FN**, actually positive but predicted negative).

$$\text{recall} = \frac{\text{TP}}{\text{TP}+\text{FN}} \;\; :: \; \; \text{False Positive Rate}$$

To produce a curve, many points are connected together. The ROC curve is produces by sorting the list of (FPR,TPR) pairs and then plotting them from the lower left corner of of the figure as a series of connecting line segments. Technically, there's no information between those points, so the optimistic curve will go straight up from each point and then straight right to the next point, the pessimistic curve will go right from each point then up, or the average curve, which interpolates. Single discrete classifiers like decision trees produce a single confusion matrix, which corresponds to a single (fpr,tpr) pair, and a curve is built from an ensemble of this discrete classifier. 

Probabilistic classifiers, however, do not make the binary predictions needed for a confusion matrix. Rather, they predict the probability that an instance will fall into a specific class. However, these probabilities can be converted to discrete values by applying a decision threshold, where probabilities below the threshold are treated as predicted negative and probabilities above the threshold are predicted positive. The curve is generated by choosing a range of thresholds and calculating the new confusion matrix values for each threshold value.  

Fawcett discusses several other methods for producing curves from single discrete classifiers. With decision trees, one can look into the model and use the class proportions calculated at each node and aggregated. Alternatively,  an ensemble of classifiers can be bootstrapped together. 

I tried to manually implement the ROC point generation algorithm that Fawcett includes in his paper, but I ran into numerous difficulties coercing types. Even in the current implementation, I must have made a mistake somewhere that I haven't been able to debug out yet as the ROC curves built with my attempt hit a wall at $0.77$ AUC and tend to have a nearly identical form. I assume I am not correctly handling slight variations in the outputs from different sklearn classifier implementations. I will continue to work on this as I would love to put a beautiful ROC curve generator function in my toolbox, or better yet, contribute code that generates a beautiful ROC curve to the sklearn project. 

|---  Base Clf ---|--- Feature Eng. ---|--- Scaling ---|--- Balancing ---|--- Best Params ---|--- Recall ---|--- Precision ---|
| :--------------: | :-------: | :--: | :-------------: |
| Logistic | None | None | SMOTE |  |  |  |
| Logistic |  |  |  |  |
| Logistic |  |  |  |  |
| Logistic |  |  |  |  |
|  |  |  |  |  |
|  |  |  |  |  |
|  |  |  |  |  |  |  |
|  |  |  |  |  |  |  |
|  |  |  |  |  |  |  |
|  |  |  |  |  |  |  |
| AdaBoost |  |  |  |  |  |  |
| AdaBoost |  |  |  |  |  |  |
| AdaBoost |  |  |  |  |  |  |


### Preprocessing Dimensions

The data description provided by UCI indicates that defined values for $\text{EDUCATION}$ and $\text{MARRIAGE}$ are $[1,2,3,4]$ and $[1,2,3]$ respectively. However, $\text{EDUCATION}$ includes values $[0,4,5]$ and $\text{MARRIAGE}$ includes values of $[0]$. I explored simply dropping these values, 

As a general convention in my code, the bold strings itemized below are included in variable names (separated by underscores) of data_variables received the corresponding preprocessing steps.  

**Undefined Value Processing**
* **NA**: The undefined values are replaced with NA values and will be imputed at model building time.
* * **no_NA**: Rows containing undefined values are simply dropped.

**Feature Engineering**
* **d0**: dummy variables for columns ['SEX', 'EDUCATION','MARRIAGE','PAY_0', 'PAY_2','PAY_3','PAY_4','PAY_5', 'PAY_6']
* **d1**: dummy variables for columns ['SEX', 'EDUCATION','MARRIAGE'] 

**Class Balancers**
* **sm**: SMOTE 
* **as**: AdaSyn  
* **st**: SMOTE+Tomek 


cc_df_d0
cc_df_d1
cc_df_d1_NA
cc_df_d1_no_NA :: classes_d1_no_NA
cc_df_no_NA :: classes_no_NA

fancyimpute.MICE().complete(data matrix)

### Scikit-Learn Exploration:

The number of possible preprocessing dimensions quickly creates a large number of combinations to try. It quickly became apparent that a coherent framework would be necessary to both accelerate development of new models, as well as reduce the space for possible variations between models. I have been using Python's sklearn module (short for Scikit-Learn, part of the SciPy scientific computing stack) in school for a couple years now, but I haven't taken the time to really dig in and study the ecosystem. The developers that designed sklearn saw that this would be a massive project and put a lot of thought into designing the API so that the project would both be usable and maintainable.  They set the design principles that all objects should share a consistent interface, only learning algorithms will be represented by classes, it should be simple to inspect an object's parameters, default values should be reasonable (i.e. the value that generally performs best given other parameters), and wherever possible, objects should be modular so that they can be assembled to perform sequential or parallel tasks.

Sklearn's consistency is largely a result of its interfaces. 
* The **estimator interface** requires that all sklearn objects must implement a **fit()** method for training a learning algorithm and a **constructor** to initialize an instance of an estimator with hyperparameters.  
* The **predictor interface** builds on estimators by requiring that estimators implement a **predict()** method that allows a fitted estimator instance

A major hurdle was figuring out how to include preprocessing steps without introducing data leakage that would contaminate the testing folds with information from the training folds, tainting the cross validation results. I found a native solution to this problem in sklearn's Pipeline interface, although, ultimately I wasn't able to get the (allegedly) fully sklearn-compliant library Imbalanced-learn$^{[ImbLearnBIB]}$ to properly apply class balancing in a pipeline. This caused a number of headaches and was a very difficult bug to diagnose, but by the 6th implementation of a manual cross-validator, I was able to integrate both class balancing and simulate a pipeline to apply a normalizing step (although I just discovered that the Imbalanced-Learn library polluted the namespace and caused the incorrect pipeline implementation to be called, resulting in an aggregate of 8+ hours of high-load computing that produced results that weren't significantly different from other runs).

Despite setbacks, bugs, redesigns, and workarounds, I was pleased with the visualizations produced by my framework, as well as the performance of the AdaBoost and RandomForest classifiers it produced. 

The general process:

1. Determine best model parameters and preprocessing steps for use with a version of a data set (semi-automated):
    1. Make a parameter grid containing a dictionary of hyperparameters and hyperparameter values to try.
    2. Select a combination of preprocessing steps and put them in a pipeline.
    3. Perform a GridSearch over the paramater grid using the preprocessing pipeline. Because I was unable to use the pipeline for class balancing, I made sample testing and training sets that are balanced by the methods explored in this paper. Fit the GridSearch using the training data and then check for overfitting by comparing the recall_scores and precision_scores produced by predictions from the training set and corresponding testing set. (Tip: save the estimator returned by GridSearchCV, it will save you several minutes if you want to use it again.)
    4. Visualize the performance over the hyperparameter space. If it there are promising trends that extend out of the selected hyperparameter window, adjust the the parameter grid and run another GridSearch. When the scores stop varying significantly with hyperparameter values, select the least complex, quickest model that meets the chosen score threshold.
2. Instantiate a new classifier with the selected hyperparameters.
3. Pass that new classifier to my evaluator function with a corresponding data set and corresponding label set, enter the desired class balancer algorithm, the scaler function, and descriptors to use in graphs. 

### Design Process

### Results



[ImbLearnBIB] Lemaitre, Guillaume, Fernando Nogueira, and Christos K. Aridas. "Imbalanced-learn: A Python Toolbox to Tackle the Curse of Imbalanced Datasets in Machine Learning." Journal of Machine Learning Research, 17th ser., 18, no. 18 (January 2017): 1-5.